In [1]:
# Interactive plot
from ipycanvas import Canvas

def hsv_to_rgb(h: float, s: float = 0.7, v: float = 1.0):
    h = h % 1.0
    r = max(0.0, min(1.0, abs(6.0 * h - 3.0) - 1.0))
    g = max(0.0, min(1.0, 2.0 - abs(6.0 * h - 2.0)))
    b = max(0.0, min(1.0, 2.0 - abs(6.0 * h - 4.0)))
    
    r = r * s + 1 - s
    g = g * s + 1 - s
    b = b * s + 1 - s
    
    return r * v, g * v, b * v

def rgb_to_str(r: float, g: float, b: float):
    r = hex(int(r * 255))[2:]
    g = hex(int(g * 255))[2:]
    b = hex(int(b * 255))[2:]
    if len(r) < 2:
        r = "0" + r
    if len(g) < 2:
        g = "0" + g
    if len(b) < 2:
        b = "0" + b
    return "#" + r + g + b

def hsv_to_str(h: float, s: float = 0.7, v: float = 1.0):
    r, g, b = hsv_to_rgb(h, s, v)
    return rgb_to_str(r, g, b)

In [2]:
from typing import Any
from enum import Enum

from gekko import GEKKO

BoxType = tuple[float, float, float, float]
InputModule = tuple[BoxType, list[BoxType], list[BoxType], list[BoxType], list[BoxType]]
OptionalMatrix = dict[int, dict[int, float]]
Hyperedge = tuple[float, list[int]]
Hypergraph = list[Hyperedge]

max_ratio = 4

def SMAX(x, y, sroot: GEKKO, tau = 0.01):
    return 0.5 * (x + y + sroot( (x - y)**2 + 4 * tau * tau ))
    
def THIN(w, h = 1):
    return 2*w*h/(w*w + h*h)

class Cardinal(Enum):
    NORTH = 0
    SOUTH = 1
    EAST = 2
    WEST = 3

class ModelModule:
    def __init__(self, 
                 x: list[Any], 
                 y: list[Any], 
                 w: list[Any], 
                 h: list[Any], 
                 gekko: GEKKO, 
                 trunk: BoxType,
                 die_width: float,
                 die_height: float):
        self.N: list[int] = []
        self.S: list[int] = []
        self.E: list[int] = []
        self.W: list[int] = []
        self.x: list[Any] = x
        self.y: list[Any] = y
        self.w: list[Any] = w
        self.h: list[Any] = h
        self.c: int = 0
        self.dw: float = die_width
        self.dh: float = die_height
        self.area = gekko.Intermediate(0)
        self.xsum = gekko.Intermediate(0)
        self.ysum = gekko.Intermediate(0)
        self.set_trunk(gekko, trunk)
        
    def set_trunk(self, gekko: GEKKO, trunk: BoxType) -> None:
        assert self.c == 0, "!"
        self.c = 1
        self._define_vars(gekko, trunk)
    
    def add_rect_north(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.N.append(self.c)
        self.c += 1
        xv, yv, wv, hv = self._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(yv == self.y[0] + 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(xv >= self.x[0] - 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(xv <= self.x[0] + 0.5 * self.w[0] - 0.5 * wv)
        
        return xv, yv, wv, hv
    
    def add_rect_south(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.S.append(self.c)
        self.c += 1
        xv, yv, wv, hv = self._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(yv == self.y[0] - 0.5 * self.h[0] - 0.5 * hv)
        gekko.Equation(xv >= self.x[0] - 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(xv <= self.x[0] + 0.5 * self.w[0] - 0.5 * wv)
        
    def add_rect_east(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.E.append(self.c)
        self.c += 1
        xv, yv, wv, hv = self._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(xv == self.x[0] + 0.5 * self.w[0] + 0.5 * wv)
        gekko.Equation(yv >= self.y[0] - 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(yv <= self.y[0] + 0.5 * self.h[0] - 0.5 * hv)
        
        return xv, yv, wv, hv
        
    def add_rect_west(self, gekko: GEKKO, rect: BoxType) -> None:
        assert self.c > 0, "!"
        self.W.append(self.c)
        self.c += 1
        xv, yv, wv, hv = self._define_vars(gekko, rect)
        
        # Keep the box attached
        gekko.Equation(xv == self.x[0] - 0.5 * self.w[0] - 0.5 * wv)
        gekko.Equation(yv >= self.y[0] - 0.5 * self.h[0] + 0.5 * hv)
        gekko.Equation(yv <= self.y[0] + 0.5 * self.h[0] - 0.5 * hv)
        
        return xv, yv, wv, hv
        
    def _define_vars(self, gekko: GEKKO, rect: BoxType) -> tuple[Any, Any, Any, Any]:
        var_x = gekko.Var(lb = 0, ub = self.dw)
        var_y = gekko.Var(lb = 0, ub = self.dh)
        var_w = gekko.Var(lb = 0, ub = self.dw)
        var_h = gekko.Var(lb = 0, ub = self.dh)
        var_x.value = [rect[0]]
        var_y.value = [rect[1]]
        var_w.value = [rect[2]]
        var_h.value = [rect[3]]
        self.x.append(var_x)
        self.y.append(var_y)
        self.w.append(var_w)
        self.h.append(var_h)
        
        self.area += var_w * var_h
        self.xsum += var_x * var_w * var_h
        self.ysum += var_y * var_w * var_h
        
        # The box must stay inside the dice
        gekko.Equation(var_x - 0.5 * var_w >= 0)
        gekko.Equation(var_y - 0.5 * var_h >= 0)
        gekko.Equation(var_x + 0.5 * var_w <= self.dw)
        gekko.Equation(var_y + 0.5 * var_h <= self.dh)
        
        # The ratio cannot exceed a maximum value
        gekko.Equation(THIN(var_w, var_h) >= THIN(max_ratio))
        
        return var_x, var_y, var_w, var_h
    
class Model:
    """GEKKO model with variables"""
    gekko: GEKKO
    
    M: list[ModelModule]
    dw: float # Die width
    dh: float # Die height

    # Model variables
    # (without accurate type hints because GEKKO does not have type hints yet)
    x: list[list[Any]]
    y: list[list[Any]]
    w: list[list[Any]]
    h: list[list[Any]]
        
    def define_module(self, trunk_box: BoxType) -> int:
        x = []
        y = []
        w = []
        h = []
        m = ModelModule(x, y, w, h, self.gekko, trunk_box, self.dw, self.dh)
        self.M.append(m)
        self.x.append(x)
        self.y.append(y)
        self.w.append(w)
        self.h.append(h)
        return len(self.M) - 1
    
    def add_rect(self, m: int, box: BoxType, direction: Cardinal) -> None:
        call = self.M[m].add_rect_west
        if direction is Cardinal.NORTH:
            call = self.M[m].add_rect_north
        elif direction is Cardinal.SOUTH:
            call = self.M[m].add_rect_south
        elif direction is Cardinal.EAST:
            call = self.M[m].add_rect_east
        xv, yv, wv, hv = call(self.gekko, box)
        return len(self.M) - 1
    
    def __init__(self, 
                 M: list[InputModule], 
                 A: list[float], 
                 X: OptionalMatrix, 
                 Y: OptionalMatrix, 
                 W: OptionalMatrix, 
                 H: OptionalMatrix,
                 die_width: float,
                 die_height: float,
                 Ω: Hypergraph):
        assert len(M) == len(A), "M and A need to have the same length!"
        
        self.M = []
        self.x = []
        self.y = []
        self.w = []
        self.h = []
        self.dw = die_width
        self.dh = die_height
        
        """Constructs the GEKKO object and initializes the model"""
        self.gekko = GEKKO(remote=False)
        
        # Variable definition
        for (trunk, N, S, E, W) in M:
            m = self.define_module(trunk)
            for box in N:
                self.add_rect(m, box, Cardinal.NORTH)
            for box in S:
                self.add_rect(m, box, Cardinal.SOUTH)
            for box in E:
                self.add_rect(m, box, Cardinal.EAST)
            for box in W:
                self.add_rect(m, box, Cardinal.WEST)
        
        # Minimal area requirements
        for m in range(0, len(A)):
            self.gekko.Equation(self.M[m].area >= A[m])
        
        # No Inter-Module Intersection
        for m in range(0, len(A)):
            for n in range(m+1, len(A)):
                for i in range(0, self.M[m].c):
                    for j in range(0, self.M[n].c):
                        t1 = (self.x[m][i] - self.x[n][j])**2 - 0.25 * (self.w[m][i] + self.w[n][j])**2
                        t2 = (self.y[m][i] - self.y[n][j])**2 - 0.25 * (self.h[m][i] + self.h[n][j])**2
                        self.gekko.Equation(SMAX(t1, t2, self.gekko.sqrt) >= 0)
        
        # Objective function
        obj = self.gekko.Intermediate(0)
        for (ω, S) in Ω:
            centroid_x = self.gekko.Intermediate(0)
            centroid_y = self.gekko.Intermediate(0)
            for i in S:
                centroid_x += self.M[i].xsum / self.M[i].area
                centroid_y += self.M[i].ysum / self.M[i].area
            for i in S:
                m = self.M[i]
                obj += ω * ω * ((m.xsum / m.area)**2 + (m.ysum / m.area)**2)
        self.gekko.Obj(obj)
                
        
    def interactive_draw(self, canvas_width = 500, canvas_height = 500):
        xmin = -1
        ymin = self.dh + 1
        xmax = self.dw + 1
        ymax = -1
        def transform(x,y):
            return (x - xmin) / (xmax - xmin) * canvas_width, (y - ymin) / (ymax - ymin) * canvas_height
        
        canvas = Canvas(width=canvas_width, height=canvas_height)
    
        canvas.fill_style = "#000"
        canvas.fill_rect(0, 0, canvas_width, canvas_height)
        
        for i in range(0, len(self.M)):
            m = self.M[i]
            hue = i / len(self.M)
            for j in range(0, len(m.x)):
                a = m.x[j][0] - 0.5 * m.w[j][0]
                b = m.y[j][0] - 0.5 * m.h[j][0]
                c = m.x[j][0] + 0.5 * m.w[j][0]
                d = m.y[j][0] + 0.5 * m.h[j][0]
                
                p1 = transform(a,b)
                p2 = transform(c,d)
                
                canvas.fill_style = hsv_to_str(hue) + "90"
                canvas.fill_rect(p1[0], p1[1], p2[0] - p1[0], p2[1] - p1[1])
        return canvas
                
    def solve(self):
        self.gekko.solve(disp=True)

In [3]:
b1: BoxType = ((2,4,3,5), [(1.5,7,2,1)], [], [], [])
b2: BoxType = ((2,2,4,4), [], [], [], [])
m = Model([b1, b2], [5, 16], None, None, None, None, 7, 7, [(1, [0,1])])
m.interactive_draw()

Canvas(width=500)

In [4]:
m.solve()

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  33
   Intermediates:  9
   Connections  :  0
   Equations    :  32
   Residuals    :  23
 
 Number of state variables:    33
 Number of total equations: -  22
 Number of slack variables: -  21
 ---------------------------------------
 Degrees of freedom       :    -10
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclips

In [5]:
m.interactive_draw()

Canvas(width=500)

In [6]:
(m.x, m.y, m.w, m.h)

([[[3.4283794759], [2.9832195967]], [[1.2690278248]]],
 [[[0.93597986018], [2.8079549282]], [[3.1520191945]]],
 [[[1.7806515324], [0.89033132892]], [[2.5380556353]]],
 [[[1.8719596546], [1.8719904814]], [[6.3040383619]]])